# Some facts from <a href="https://docs.python.org/3/tutorial/"> official Python tutorial </a>
*"Flying by instructions" is sometimes the most time-efficient way of doing things, but it's one-off approach.*

*When you start using tool daily you should devote some time to its general exploration.*

*For me as a C/C++ programmer the most interesting are Python features that do not have direct analog in C/C++*

## Strings and string literals

#### Regular vs raw string
*NB: String concatenation by placing **literals** one after another*

In [7]:
print('C:\some\\name')
print(r'C:\some\name')

# Single \ at the end of raw string needs workaround
print('C:\some\\name\\')
print(r'C:\some\name' '\\')
# or
rs = (r'C:\some\name' # to concatenate literals over line break put them in parentheses
     '\\')
print(rs)

C:\some\name
C:\some\name
C:\some\name\
C:\some\name\
C:\some\name\


#### \ at the beginning of tripple-quoted string removes leading empty line

In [5]:
# With
print(40 * '-')
print("""\
Usage: thingy [OPTIONS]
     -h                        Display this usage message
     -H hostname               Hostname to connect to
""")

# and without
print(40 * '-')
print("""
Usage: thingy [OPTIONS]
     -h                        Display this usage message
     -H hostname               Hostname to connect to
""")


----------------------------------------
Usage: thingy [OPTIONS]
     -h                        Display this usage message
     -H hostname               Hostname to connect to

----------------------------------------

Usage: thingy [OPTIONS]
     -h                        Display this usage message
     -H hostname               Hostname to connect to



#### Slicing is more forgiving than direct indexing

In [10]:
'Python'[41:42]

''

In [12]:
'Python'[41]

IndexError: string index out of range

### Formatting Python strings for C/C++ programmers
#### f"", string.format(), printf-style, string.Template formatting
*val!s ≡ str(val), val!r ≡ repr(val), val!a ≡ ascii(val)*

In [22]:
name = 'Fred'
city = 'Düsseldorf'
print(f'He said his name is {name!r} and he is from {city!a}')
print('He said his name is {1!r} and he is from {0!a}'.format(city, name))
print('He said his name is %(name)r and he is from %(city)a' % {'name': name, 'city': city})

from string import Template
s = Template('He said his name is $name and he is from $city')
print(s.substitute(name = repr(name), city = ascii(city)))

print(s.safe_substitute(city = ascii(city)))

He said his name is 'Fred' and he is from 'D\xfcsseldorf'
He said his name is 'Fred' and he is from 'D\xfcsseldorf'
He said his name is 'Fred' and he is from 'D\xfcsseldorf'
He said his name is 'Fred' and he is from 'D\xfcsseldorf'
He said his name is $name and he is from 'D\xfcsseldorf'


#### Precision means overall number of digits rather than number of digits after the decimal point 
#### in all cases but printf-style formatting

In [20]:
width = 10
precision = 3
val = 12.34567
print(f'value:{val:{width}.{precision}}')
print('value:{:10.3}'.format(val)) #cannot make it work with vars
print('value:%(val)10.1f' % {'val': val}) # NB: precision 


value:      12.3
value:      12.3
value:      12.3


#### Example of date formatting

In [24]:
from datetime import datetime

today = datetime(year = 2023, month = 1, day = 28)
print(f'{today:%B %d, %Y}')
print('{:%B %d, %Y}'.format(today))


January 28, 2023
January 28, 2023


#### Use = to show variable name
*NB: whitespaces are preserved*

In [26]:
print(f'{today=:%B %d, %Y}')

foo = 'bar'
print(f'{foo = }')

today=January 28, 2023
foo = 'bar'


#### Integer base

In [34]:
n = 1000
print(f'{n:#0x} {n:#b} {n:#o}')
print('{0:#0x} {0:#b} {0:#o}'.format(n))
print('%(n)#0xd %(n)#od' % {'n': n}) #could not find binary format code for this example

0x3e8 0b1111101000 0o1750
0x3e8 0b1111101000 0o1750
0x3e8d 0o1750d


#### Alterate quotes for correct result, backslashes are not allowed

In [34]:
a = {'x': ord('x'), 'y': ord('y')}
newline = ord('\n')

print(f'{a["x"] = }')
print(f'new line code: {newline}')
print(f'"a" code: {ord("a")}')

a["x"] = 120
new line code: 10
"a" code: 97


In [35]:
# But not:
f'{a[\'x\'] = }'

SyntaxError: f-string expression part cannot include a backslash (1603011531.py, line 2)

In [36]:
# or
f"new line code: {ord('\n')}"

SyntaxError: f-string expression part cannot include a backslash (710286865.py, line 2)

#### Formatted string literals cannot be used as docstrings even if they do not include expressions

In [37]:
def fun0():
    'A doc string'
    pass

def fun1():
    f'Not a doc string'
    pass

n = 8
def fun2():
    'Not a doc string {}'.format(n)
    pass

print(fun0.__doc__)
print(fun1.__doc__)
print(fun2.__doc__)

A doc string
None
None


In [45]:
num = 1234567.89
print('{}'.format(num))
print('{:n}'.format(num))

1234567.89
1.23457e+06


## switch-case 
#### match-case in Python is more complicated and more useful than switch-case in C/C++

*case _: ≡ default:*

In [38]:
def error2str(err):
    match err:
        case 400:
            return 'Bad request'
        case 401 | 403 | 404:
            return 'Not allowed'
        case _:
            return 'Something is wrong with the internet'
        
error2str(418)

'Something is wrong with the internet'

#### tuple match

In [39]:
def show_point(pt): # tuple (x, y) expected
    match(pt):
        case (0, 0): 
            print('Origin')
        case (0, y):
            print(f'y = {y}')
        case (x, 0):
            print(f'x = {x}')
        case (x, y):
            print(f'x = {x}, y = {y}')
        case _:
            raise ValueError('Not a point')

show_point((0, 0))
show_point((2.5, 0))
show_point((0, -0.1))
show_point((-1, 4))
show_point((1, 2, 3))

Origin
x = 2.5
y = -0.1
x = -1, y = 4


ValueError: Not a point

#### This example is taken unchnaged from the tutorial fails but can easily be fixed

In [40]:
class Point:
    x: int
    y: int

def where_is(point):
    match point:
        case Point(x=0, y=0):
            print("Origin")
        case Point(x=0, y=y):
            print(f"Y={y}")
        case Point(x=x, y=0):
            print(f"X={x}")
        case Point():
            print("Somewhere else")
        case _:
            print("Not a point")
            
p = Point()
where_is(p)
print(p.x, p.y)

Somewhere else


AttributeError: 'Point' object has no attribute 'x'

In [41]:
p = Point(1, 2)
where_is(p)

TypeError: Point() takes no arguments

In [42]:
p = Point(x = 1, y = 2)
where_is(p)

TypeError: Point() takes no arguments

#### Fixes:
***Make Point1 a regular class***

In [48]:
class Point1:
    def __init__(self, x, y):
        self.x = x
        self.y = y       


def where_is1(point):
    match point:
        case Point1(x=0, y=0):
            print("Origin")
        case Point1(x=0, y=y):
            print(f"Y={y}")
        case Point1(x=x, y=0):
            print(f"X={x}")
        case Point1():
            print("Somewhere else")
        case _:
            print("Not a point")
            
where_is1(Point1(0, 0))
where_is(Point1(0, 0))

Origin
Not a point


In [45]:
p = Point1() # fails while in match-case it's OK

TypeError: Point1.__init__() missing 2 required positional arguments: 'x' and 'y'

***Make Point2 dataclass***

In [56]:
from dataclasses import dataclass

@dataclass 
class Point2:
    x: int
    y: int
    
def where_is2(point):
    match point:
        case Point2(x=0, y=0):
            print("Origin")
        case Point2(x=0, y=y):
            print(f"Y={y}")
        case Point2(x=x, y=0):
            print(f"X={x}")
        case Point2(): # Again it's OK unlike assignment below
            print("Somewhere else")
        case _:
            print("Not a point")    
    
p = Point2(2, 3.1)

where_is2(p)
where_is2(Point2())

Somewhere else


TypeError: Point2.__init__() missing 2 required positional arguments: 'x' and 'y'

***Make Point3 NamedTuple***

In [55]:
from typing import NamedTuple

class Point3(NamedTuple):
    x: int
    y: int

def where_is3(point):
    match point:
        case Point3(x=0, y=0):
            print("Origin")
        case Point3(x=0, y=y):
            print(f"Y={y}")
        case Point3(x=x, y=0):
            print(f"X={x}")
        case Point3(): # Again it's OK unlike assignment below
            print("Somewhere else")
        case _:
            print("Not a point")
            
p = Point3(y = 1, x = 2.3)
where_is3(p)
where_is(Point3())

Somewhere else


TypeError: Point3.__new__() missing 2 required positional arguments: 'x' and 'y'

## Scopes

In [59]:
def scope_test():
    def do_local():
        spam = 'local spam'
        
    def do_arg(spam):
        spam = 'reference spam'
        
    def do_nonlocal():
        nonlocal spam
        spam = 'nonlocal spam'
        
    def do_global():
        global spam # declares var spam in global scope
        spam = 'global spam'
        
    spam = 'test spam' # NB: we are inside scope_test() scope now
    do_local()
    print('After do_local():', spam)
    do_arg(spam)
    print('After do_arg():', spam)
    do_nonlocal()
    print('After do_nonlocal():', spam)
    do_global()
    print('After do_global():', spam)
    
scope_test()

print('In global scope:', spam) 

After do_local(): test spam
After do_arg(): test spam
After do_nonlocal(): nonlocal spam
After do_global(): nonlocal spam
In global scope: global spam


## Function arg tricks

#### Default values for args

In [60]:
i = 5

def f(arg = i): # it default to current value of i rather than to i as a reference
    print(arg)
    
i = 10
print('Default: ')
f()
print('With argument: ')
f(i)

Default: 
5
With argument: 
10


In [61]:
def af(a, l = []): # default evaluates once, for mutable object it means the object is the same and undergoes consecutive changes
    l.append(a)
    return l

print(af(0))
print(af(1))
print(af(2))
print(af(3, []))

[0]
[0, 1]
[0, 1, 2]
[3]


#### Using *args and **kwargs

In [62]:
def cheeseshop(kind, *args, **kwargs): # NB: args and kwargs are still passed by value
    print('- Do you have any', kind, '?')
    print('- I\'m sorry, we are all out of', kind)
    
    i = 0
    for a in args:
        print(a)
        i+=1
        a = i
        
    print('-' * 40)
    
    for k in kwargs:
        print(k, ':', kwargs[k])
        i+=1
        kwargs[k] = i
        
arg0 = 'It\'s very runny, sir.'
arg1 = [10, 9, 8, 7]

shopkeeper = 'Michael Palin'
client     = 'John Cleese'
sketch     = 'Cheese Shop Sketch'

# passing iterable preseeded by * unfolds it
cheeseshop('Limburger', arg0, *arg1, shopkeeper = shopkeeper, client = client, sketch = sketch) 
print(arg0, arg1, shopkeeper, client, sketch)

- Do you have any Limburger ?
- I'm sorry, we are all out of Limburger
It's very runny, sir.
10
9
8
7
----------------------------------------
shopkeeper : Michael Palin
client : John Cleese
sketch : Cheese Shop Sketch
It's very runny, sir. [10, 9, 8, 7] Michael Palin John Cleese Cheese Shop Sketch


#### Positional only, positional or keyword, keyword only arguments

In [63]:
def standard_arg(arg):
    print(arg)
    
def pos_only_arg(arg, /):
    print(arg)
    
def kwd_only_arg(*, arg):
    print(arg)
    
def combined_arg(pos_only, /, standard, *, kwd_only):
    print(pos_only, standard, kwd_only)

In [64]:
# correct calls

standard_arg('std')
standard_arg(arg = 'std 1')
pos_only_arg('pos')
kwd_only_arg(arg = 'kwd')
combined_arg('pos', 'std', kwd_only = 'kwd')
combined_arg('pos', kwd_only = 'kwd', standard = 'std1')

std
std 1
pos
kwd
pos std kwd
pos std1 kwd


In [65]:
# duplicate arg
standard_arg(0, arg = 0)

TypeError: standard_arg() got multiple values for argument 'arg'

In [66]:
# Positional only arg assigned by name
pos_only_arg(arg = 0)

TypeError: pos_only_arg() got some positional-only arguments passed as keyword arguments: 'arg'

In [67]:
# Keyword only arg passed as positional
kwd_only_arg(0)

TypeError: kwd_only_arg() takes 0 positional arguments but 1 was given

#### Standard and positional args with **kwargs - solving arg name clash

In [68]:
def std_kwd(name, **kwds):
    return 'name' in kwds # always False or Error

def pos_kwd(name, /, **kwds):
    return 'name' in kwds # can have name arg

In [69]:
std_kwd('John', name = 'John')

TypeError: std_kwd() got multiple values for argument 'name'

In [71]:
std_kwd('John', **{'name': 'John'}) #another way to assign kwargs

TypeError: std_kwd() got multiple values for argument 'name'

In [72]:
pos_kwd('John', name = 'Jenny')

True

In [73]:
pos_kwd('John', **{'name': 'John'})

True

#### Unfold/unpack arguments

In [75]:
def parrot(voltage, state, action):
    print('This parrot would not', action, end = ' ')
    print('if you put', voltage, 'volts through it.', end = ' ')
    print("E's", state, '!')
    
args = (360, 'dead', 'fly')
kwargs = {'voltage': 'four million', 'action': 'beg sugar', 'state': None}

parrot(*args)
parrot(**kwargs)

This parrot would not fly if you put 360 volts through it. E's dead !
This parrot would not beg sugar if you put four million volts through it. E's None !


#### Passing by reference ('out argument')

In [76]:
def change_it(i):
    i[0] = 5
    
a = [666]
change_it(a)
print(a)

[5]


## for statement

### for (int i = 0; i < 10; ++i) { ... }

In [90]:
for i in range(10):
    print(i, end = ' ')

0 1 2 3 4 5 6 7 8 9 

#### for (int i = 1; i < 10; i += 2) { ... }

In [91]:
for i in range(1, 10, 2):
     print(i, end = ' ')

1 3 5 7 9 

#### break and continue work the same but there is else clause in for loop

In [92]:
for n in range(2, 10):
    for x in range(2, n):
        if n % x == 0:
            print('{} equals {} * {}'.format(n, x, n//x))
            break
    else: # loop ended without a break
        print(n, 'is a prime number')

2 is a prime number
3 is a prime number
4 equals 2 * 2
5 is a prime number
6 equals 2 * 3
7 is a prime number
8 equals 2 * 4
9 equals 3 * 3


## Classes and objects

### Class variables

In [78]:
class Warehouse:
    purpose = 'storage'
    region  = 'west'
    
w1 = Warehouse()
w2 = Warehouse()
w2.region = 'east'
w1.purpose = 'salvage'

print(w1.purpose, w1.region)
print(w2.purpose, w2.region)

salvage west
storage east


#### But!

In [79]:
class Dog:
    tricks = []
    
    def __init__(self, name):
        self.name = name
        
fido = Dog('Fido')
fido.tricks.append('roll over')

buddy = Dog('Buddy')
buddy.tricks.append('play dead')

print(fido.tricks)
print(buddy.tricks)

['roll over', 'play dead']
['roll over', 'play dead']


### Each value is an object and its type is stored as _class_ attribute

In [80]:
'string'.__class__

str

In [81]:
True.__class__

bool

In [82]:
5.6.__class__

float

In [83]:
1.__class__

SyntaxError: invalid decimal literal (4151937913.py, line 1)

In [85]:
(1).__class__

int

In [86]:
issubclass(bool, int)

True

In [87]:
b = False
isinstance(b, int)

True

In [88]:
isinstance(True, int)

True